In [18]:
import torch
import os
import glob
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from typing import Any
from torchsummary import summary



# Customized Dataset

In [20]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.transform = transform
        self.with_makeup_paths = glob.glob(os.path.join(root_dir,"with","*.jpg"))
        self.without_makeup_paths = glob.glob(os.path.join(root_dir,"without","*.jpg"))
    def __len__(self) -> int:
        return len(self.with_makeup_paths)
    def __getitem__(self, index) -> Any:
        with_makeup_path = self.with_makeup_paths[index]
        without_makeup_path = self.without_makeup_paths[index]
        
        with_makeup_image = Image.open(with_makeup_path)
        without_makeup_image = Image.open(without_makeup_path)
        
        if self.transform:
            with_makeup_image = self.transform(with_makeup_image)
            without_makeup_image = self.transform(without_makeup_image)
            
        return [with_makeup_image, without_makeup_image]
        

# Model

## Element of GAN

In [21]:
class ConvBlock(torch.nn.Module):
    def __init__(self, out_channels, kernel_size, stride=1, padding=0) -> None:
        super().__init__()
        self.block = torch.nn.Sequential(
            torch.nn.LazyConv2d(out_channels, kernel_size=kernel_size, stride=stride,
                            padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(),
        )
    def forward(self, x) -> torch.Tensor:
        return self.block(x)

class DeConvBlock(torch.nn.module):
    def __init__(self, out_channels, kernel_size, stride=1, padding=0, 
                 output_padding=0, activation=torch.nn.ReLU) -> None:
        super().__init__()
        self.block = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(out_channels, kernel_size=kernel_size, 
                                     stride=stride, padding=padding, output_padding=output_padding),
            torch.nn.BatchNorm2d(out_channels, inplace=True),
            activation()
        )
    def forward(self, x) -> torch.Tensor:
        return self.block(x)
    
class ResidualBlock(torch.nn.Module):
    def __init__(self, output_channels, kernel_size, stride=1, padding=0, activation=torch.nn.ReLU):
        super().__init__()
        self.block = torch.nn.Sequential(
            torch.nn.LazyConv2d(output_channels, kernel_size, stride, padding),
            torch.nn.ReflectionPad2d(1),
            torch.nn.BatchNorm2d(output_channels),
            activation(),
            torch.nn.LazyConv2d(output_channels, kernel_size, stride, padding),
            torch.ReflectionPad2d(1),
            torch.nn.BatchNorm2d(output_channels)
        )
    def forward(self, input_tensor) -> torch.Tensor:
        x = self.block(x),
        x = x + input_tensor
        return x
def replace_layers(model):
    for n, module in model.named_children():
        if len(list(module.children())) > 0:
            ## compound module, go inside it
            replace_layers(module)
        if isinstance(module, torch.nn.MaxPool2d):
            setattr(model, n, torch.nn.Conv2d)
        
class Generator(torch.nn.Module):
    def __init__(self,input_channels=3, output_channels=3, filters=64, n_blocks= 10) -> None:
        super().__init__()
        down_scale=[torch.nn.ReflectionPad2d(3),
            ConvBlock(output_channels=filters, kernel_size=7, activation=torch.nn.LeakyReLU),
            ConvBlock(out_channels=filters*2, kernel_size=3, stride=2, activation=torch.nn.LeakyReLU),
            ConvBlock(out_channels=filters*4, kernel_size=3, stride=2, activation=torch.nn.LeakyReLU)]
        for i in range(n_blocks):
            down_scale.append(ResidualBlock(output_channels=filters*4, kernel_size=3,activation=torch.nn.LeakyReLU()))
        
        up_scale = [
            DeConvBlock(out_channels=filters*2, kernel_size=3, stride=2, padding=1, output_padding=1,activation=torch.nn.LeakyReLU),
            DeConvBlock(out_channels=filters, kernel_size=3, stride=2,padding=1, output_padding=1, activation=torch.nn.LeakyReLU),
            torch.nn.ReflectionPad2d(3),
            torch.nn.LazyConv2d(out_channels=output_channels, kernel_size=7, stride=1,padding=0, activation=torch.nn.Tanh)
        ]
            
        self.down_scale = torch.nn.Sequential(*down_scale)
        self.up_scale = torch.nn.Sequential(*up_scale)

    def forward(self, x)-> torch.Tensor:
        latent_vector = self.down_scale(x)
        fake_image = self.up_scale(latent_vector)
        return fake_image

class Discriminator(torch.nn.Module):
    def __init__(self, input_channels=3, filters=64) -> None:
        super().__init__()
        layers = [
            torch.nn.LazyConv2d(out_channels=filters, kernel_size=4, stride=2, padding=1),
            torch.nn.LeakyReLU(),
            ConvBlock(out_channels=filters*2, kernel_size=4, stride=2, padding=1, activation=torch.nn.LeakyReLU),
            ConvBlock(out_channels=filters*4, kernel_size=4, stride=2, padding=1, activation=torch.nn.LeakyReLU),
            ConvBlock(out_channels=filters*8, kernel_size=4, stride=1, padding=1, activation=torch.nn.LeakyReLU),
        ]
        # Output layers
        layers.append(torch.nn.LazyConv2d(output_channels=1, kernel_size=4, stride=1, padding=1))
        self.model = torch.nn.Sequential(*layers)
    def forward(self, x)-> torch.Tensor:
        return self.model(x)

AttributeError: module 'torch.nn' has no attribute 'module'

In [19]:
sample = torch.randn(1,3,96,96)
gen = Generator()

summary(gen,(3,96,96))

TypeError: ConvTranspose2d.__init__() missing 3 required positional arguments: 'in_channels', 'out_channels', and 'kernel_size'

In [7]:
generator = Generator()

replace_layers(generator,(torch.nn.MaxPool2d, torch.nn.AdaptiveAvgPool2d), torch.nn.Identity())
print(generator)


TypeError: ConvTranspose2d.__init__() missing 3 required positional arguments: 'in_channels', 'out_channels', and 'kernel_size'

# Customized Loss

# Train

In [ ]:
root_dir = os.path.join(os.getcwd(), "dataset","gan_makeup_data_96")
transform = transforms.Compose([
    transforms.Resize([96,96]),
    transforms.ToTensor(),
    # transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

dataset = ImageDataset(root_dir, transform)
fig, axes = plt.subplots(10, 2, figsize=(10, 20))
for i in range(10):
    with_image = transforms.ToPILImage()(dataset[i][0])
    without_image = transforms.ToPILImage()(dataset[i][1])
    axes[i,0].imshow(with_image)
    axes[i,0].axis("off")
    axes[i,1].imshow(without_image)
    axes[i,1].axis("off")
plt.tight_layout()    
plt.show()

NameError: name 'os' is not defined